In [ ]:
import numpy as np

class Facet(object):
    def __init__(self, parent, aPolygons = [], boundaryID = 0, aHoles = np.empty((0,3), np.float32)):
        assert boundaryID >= 0, "Boundary ID must be greater zero (>0)"
        self.aPolygons = aPolygons
        self.aHoles = aHoles
        self.boundaryID = boundaryID
        
    def __repr__(self):
        result = repr(len(self.aPolygons))
        result += " " + repr(self.aHoles.shape[0])
        result += " " + repr(self.boundaryID) + "\n"
        for i, p in enumerate(self.aPolygons):
            result += repr(len(p)) + " "
            for j in range(len(p)):
                result += repr(p[j]) + " "
            result += "\n"
        for i in range(self.aHoles.shape[0]):
            hole = self.aHoles[i, :]
            result += "{} {} {} {}\n".format(i+1, hole[0], hole[1], hole[2])
        return result
        
    def appendPolygon(self, aPolygon, hole = None):
        self.aPolygons.append(np.array(aPolygon, dtype=np.int32))
        if not hole is None:
            hole.reshape((1, 3))
            self.aHoles = np.concatenate((self.aHoles, hole.reshape((1, 3))))

    def offsetPolygons(self, offset):
        for i in range(len(self.aPolygons)):
            self.aPolygons[i] = offset + self.aPolygons[i] 
        
class Object3D(object):
    def __init__(self):
        self.aVertices = np.empry((0, 3), dtype=np.float32)
        self.aFacets = []
        
    def __repr__(self):
        aVertices = self.vertices()
        aFacets = self.facets()
        result = "# Node count, dim count, attribute count, boundary marker {0, 1}\n"
        result += repr(aVertices.shape[0]) + " 3 0 0\n"
        result += "Vertices:\n"
        for i, v in enumerate(aVertices):
            result += repr(i+1) + " " + repr(v[0]) + " " + repr(v[1]) + " " + repr(v[2]) + "\n"
        result += "\n# Facets\n"
        result += repr(len(aFacets)) + " 1\n"
        for i, f in enumerate(aFacets):
            result += repr(f)
        return result
        
    def vertices(self):
        return self.aVertices
    
    def facets(self):
        return self.aFacets
    
class Transform3D(Object3D):
    def __init__(self, child):
        self.child = child
        
    def facets(self):
        return self.child.facets()
        
class Translation(Transform3D):
    def __init__(self, child, translationVector):
        assert not (child is None), "Child must be given at construction."
        self.child = child
        self.v = translationVector
        
    def vertices(self):
        aVertices = np.empty_like(self.child.aVertices)
        for i in range(aVertices.shape[0]):
            aVertices[i, :] = self.v + self.child.aVertices[i, :]
        return aVertices
        
class PiecewiseLinearComplex(Object3D):
    def __init__(self, aVertices = None, aFacets = []):
        if aVertices is None:
            self.aVertices = np.empty((0, 3), dtype=np.float32)
        else:
            self.aVertices = aVertices
        self.aFacets = aFacets
        
    def __repr__(self):
        result = "# Node count, dim count, attribute count, boundary marker {0, 1}\n"
        result += repr(self.aVertices.shape[0]) + " 3 0 0\n"
        for i, v in enumerate(self.aVertices):
            result += repr(i+1) + " " + repr(v[0]) + " " + repr(v[1]) + " " + repr(v[2]) + "\n"
        result += "\n# Facet count, boundary markers\n"
        result += repr(len(self.aFacets)) + " 1\n"
        for i, f in enumerate(self.aFacets):
            result += "# Facet {}\n".format(i+1)
            result += repr(f)
        return result
        
    def appendVertices(self, aVertices):
        offset = self.aVertices.shape[0]
        self.aVertices = np.concatenate((self.aVertices, aVertices))
        return offset
    
    def appendFacets(self, aFacets):
        self.aFacets = np.concatenate((self.aFacets, aFacets))
        
    def offsetPolygons(self, offset):
        for i, f in enumerate(self.aFacets):
            f.offsetPolygons(offset)
        
        
class Box3D(PiecewiseLinearComplex):
    # Create a box representation for TetGen. The box's front 
    # face (baffle) lies in the x-y-plane. The optional parameters
    # center_x and center_y define the acoustic center of the baffle
    # which is placed at the x-y-planes origin. If not specified,
    # the geometric center of the baffle is chosen as the acoustic
    # center.
    def __init__(self, w, h, d, boundaryID = 0):
        aVertices = np.empty((8, 3), dtype=np.float32)
        
        aVertices[0, :] = np.array([0.0, 0.0, 0.0], dtype=np.float32)
        aVertices[1, :] = np.array([w,   0.0, 0.0], dtype=np.float32)
        aVertices[2, :] = np.array([w,   h,   0.0], dtype=np.float32)
        aVertices[3, :] = np.array([0.0, h,   0.0], dtype=np.float32)
        
        aVertices[4, :] = np.array([0.0, 0.0, -d], dtype=np.float32)
        aVertices[5, :] = np.array([w,   0.0, -d], dtype=np.float32)
        aVertices[6, :] = np.array([w,   h,   -d], dtype=np.float32)
        aVertices[7, :] = np.array([0.0, h,   -d], dtype=np.float32)
        
        
        # Faces is a python array of numpy integer arrays.
        aFacets = [Facet(self, [np.array([1, 2, 3, 4], np.int32)], boundaryID = boundaryID), # front
                   Facet(self, [np.array([5, 6, 7, 8], np.int32)], boundaryID = boundaryID), # back
                   Facet(self, [np.array([1, 4, 8, 5], np.int32)], boundaryID = boundaryID), # left
                   Facet(self, [np.array([2, 6, 7, 3], np.int32)], boundaryID = boundaryID), # right
                   Facet(self, [np.array([4, 3, 7, 8], np.int32)], boundaryID = boundaryID), # top
                   Facet(self, [np.array([1, 5, 6, 2], np.int32)], boundaryID = boundaryID)] # bottom
        super(Box3D, self).__init__(aVertices, aFacets)

    def __str__(self):
        w = self.width()
        h = self.height()
        d = self.depth()
        result = "Box:" + "\n"
        result += "  width    = " + repr(w) + "\n"
        result += "  height   = " + repr(h) + "\n"
        result += "  depth    = " + repr(d) + "\n"
        return result
    
    def width(self):
        return self.aVertices[1, 0] - self.aVertices[0, 0]

    def height(self):
        return self.aVertices[2, 1] - self.aVertices[0, 1]
    
    def depth(self):
        return self.aVertices[0, 2] - self.aVertices[4, 2]
        
    def front(self):
        return self.aFacets[0]
    
    def back(self):
        return self.aFacets[1]
    
    def left(self):
        return self.aFacets[2]
    
    def right(self):
        return self.aFacets[3]
    
    def top(self):
        return self.aFacets[4]
    
    def bottom(self):
        return self.aFacets[5]
    
    
class Disk2D(PiecewiseLinearComplex):
    def __init__(self, r, s = 12, boundaryID = 0):
        aVertices = np.empty((s, 3), dtype=np.float32)
        dAlpha = 2.0 * np.pi / s
        for i in range(s):
            alpha = i * dAlpha
            aVertices[i, :] = np.array([r * np.cos(alpha), r * np.sin(alpha), 0.0], dtype=np.float32)
        facet = Facet(self, [np.arange(1, s+1, dtype=np.int32)], boundaryID = boundaryID)
        super(Disk2D, self).__init__(aVertices, [facet])
            
    
# Dimensions for Fostex FF125WK bass reflex box.
# These are the inner dimensions.
box = Box3D(0.142, 0.296, 0.218, 1)

disk = Disk2D(0.052, 30, 2)
diskCenter = np.array((box.width() / 2.0, box.height() - 0.070, 0.0), np.float32)
translationDisk = Translation(disk, diskCenter)

offset = box.appendVertices(translationDisk.vertices())
disk.offsetPolygons(offset)
baffle = box.front()
baffle.appendPolygon(disk.aFacets[0].aPolygons[0], diskCenter)
box.appendFacets(disk.facets())
print repr(box)
file = open('SimpleSpeaker.poly', 'w')
file.write(repr(box))
file.close()
    